In [1]:
import maboss
import ginsim
import pandas as pd 
import numpy as np
import mygene
import os
import shutil
import ast
from scipy.stats import kruskal
from statsmodels.stats.multitest import multipletests
from collections import defaultdict
from pathlib import Path



from create_generic_models.create_generic_patients_cfgs import create_generic_patients_cfg_bnd_validation
from create_generic_models.update_phenotypes_generic_models import generic_models_update_phenotypes


# from create_person_models.tailor_cfgs_patients_gene import personalized_patients_genes_cfgs_validation
from pre_process_data.pre_process_genes import create_table_rna_seq_patients, create_table_proteins_patients

from identification_patients.validation_get_patients_ids import get_patients_valid

from pre_process_data.tcga_preprocess_data import pre_process_tcga_data

from create_person_models.tailor_cfgs_patients_gene import personalized_patients_genes_cfgs, personalized_patients_proteins_cfgs
from create_person_models.tailor_bnd_cnv import tailor_bnd_cnv_validation

from create_person_models.tailor_bnd_tsg_onco_mutations import tailor_bnd_mutat_validation

from MaBoSS_simulation.maboss_phenotype_patient import compute_phenotype_table, compute_phenotype_mean_group_validation
from stats.stats_proba import compute_kruskal_test_means_validation

In [2]:
# create directory where the models and results will be saved

type_models = 'proteins_models'

src_dir_generic_models = f'models/prostate/generic/{type_models}'
folder_generic_models = f'validation/prostate/{type_models}/generic_models'
# folder_generic_models_bnd = f'validation/prostate/{type_models}/generic_models'
folder_pers_models = f'validation/prostate/{type_models}/personalized_models'

folder_save_results = f"validation/prostate/{type_models}/results/phenotype_distribution/phenotype_table"
dest_base_dir = f"validation/prostate/{type_models}/results/phenotype_group_means"

directories = [
    Path(src_dir_generic_models).parent,
    Path(folder_generic_models),
    Path(folder_pers_models),
    Path(folder_save_results),
    Path(dest_base_dir),
]

# Create them
for directory in directories:
    directory.mkdir(parents=True, exist_ok=True)



# copy the generic model from the models to validation generic models
for filename in os.listdir(src_dir_generic_models):
    src_file = os.path.join(src_dir_generic_models, filename)
    dst_file = os.path.join(folder_generic_models, filename)
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)


In [3]:
tissue = 'Prostate'

# for the model
phenotype_interest = ["Proliferation","Invasion","DNA_Repair","Migration","Apoptosis"]



In [4]:
# Import data
phenotype_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_phenotypes.csv')
genes_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_genes_illumina.csv', sep='\t')
cnv_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_cnv_gistic2.csv',sep='\t')

proteins_data = pd.read_csv('data/TCGA_data/prostate/TCGA_PRAD_proteins_RPPA.csv',sep='\t')

# keep all montagud nodes
montagud_data = (
    pd.read_csv('data/Montagud_inter_nodes_data.csv', header=1)
    .loc[:, ['Target node', 'Interaction type', 'Source']])

# Create list of genes of interest (in Montagud data)
montagud_nodes = list(set(montagud_data['Target node'].tolist() + montagud_data['Source'].tolist()))
montagud_nodes = [node for node in montagud_nodes if node != '0/1']
montagud_nodes = [node.upper() for node in montagud_nodes if isinstance(node, str)]
montagud_nodes = [node.replace('_', "") for node in montagud_nodes]


# remove all '_'
montagud_nodes = ['MEK1' if x == 'MEK1_2' else x for x in montagud_nodes]
montagud_nodes = ['TSC1' if x == 'TSC1_2' else x for x in montagud_nodes]
montagud_nodes = ['MAP3K1' if x == 'MAP3K1_3' else x for x in montagud_nodes]
montagud_nodes = ['CHK1' if x == 'CHK1_2' else x for x in montagud_nodes]


montagud_nodes.append('MEK2')
montagud_nodes.append('TSC2')
montagud_nodes.append('MAP3K3')
montagud_nodes.append('CHK2')


# TO THINK ABOUT - do i really need to remove these two genes??
# to_remove = ['FUSED_EVENT', 'AR_ERG']
# montagud_nodes = [node for node in montagud_nodes if node not in to_remove]

AKT: how much AKT protein is present.
AKT_PS473: how much of the protein is in the activated or modified form.

how to do with the activated form of the protein ?  -> remove the activated form

In [5]:

phenotype_data_filtered = phenotype_data[['sampleID','gleason_score']]


In [6]:
# create 3 groups: gleason score of 6, gleason score of 7, and of gleason score of > 8

group_0 = [6]
group_1 = [7]
group_2 = [8, 9, 10]

conditions = [
phenotype_data_filtered["gleason_score"].isin(group_0),
phenotype_data_filtered["gleason_score"].isin(group_1),
phenotype_data_filtered["gleason_score"].isin(group_2),

]
choices = ['low_aggressive', 'middle_aggressive', 'high_aggressive']

phenotype_data_filtered.loc[:, "Gleason_group"] = np.select(
conditions, choices, default=""
)



/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_27579/1549429481.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenotype_data_filtered.loc[:, "Gleason_group"] = np.select(


In [7]:
sampled_df = phenotype_data_filtered.groupby("Gleason_group", group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 30), random_state=42)
)
patients_id = list(sampled_df['sampleID'])


/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_27579/3074495877.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = phenotype_data_filtered.groupby("Gleason_group", group_keys=False).apply(


In [8]:
# Pre-process proteins data (only id in the patients id list  and proteins in montagud list)

proteins_data['sample'] = proteins_data['sample'].str.rsplit('-', n=2).str[0]
proteins_data['sample'] = proteins_data['sample'].str.upper()
proteins_data["sample"] = proteins_data["sample"].str.replace("_", "", regex=False)
proteins_data["sample"] = proteins_data["sample"].str.replace("-", "", regex=False)

pattern = '|'.join(montagud_nodes)  

# Filter rows where 'sample' contains any name from the list
proteins_data = proteins_data[proteins_data['sample'].str.contains(pattern, case=False, na=False)]
proteins_data = proteins_data.dropna(how='all', subset=proteins_data.columns[1:])

mods = ['_PS', '_PT', '_PY']
proteins_data = proteins_data[
    ~proteins_data['sample'].apply(lambda p: any(mod in p for mod in mods))
]
proteins_list = list(proteins_data['sample'])



# Keep only the proteins present in the montagud list

proteins_data_col = list(proteins_data.columns)
common_col = list(set(proteins_data_col) & set(patients_id))
col_keep = ['sample'] + common_col
proteins_data_filtered = proteins_data[col_keep]


df_melted_protein = proteins_data_filtered.melt(
    id_vars=["sample"],       # columns to keep fixed
    var_name="samples_id",         # name for the variable column (sample IDs)
    value_name="expression_value"  # name for the values
)

df_melted_protein['sample'] = df_melted_protein['sample'].str.split('|').str[0] 

df_melted_protein = df_melted_protein.rename(
    columns={
        "samples_id": "model_id",
        "sample": "protein_symbol",
        "expression_value": "rsem_tpm",
    }
)
df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].str.upper()


def replace_with_base_name(protein_name):
    for base in montagud_nodes:
        if protein_name.startswith(base):
            return base
    return protein_name  # if no match found, keep original

# Assuming your dataframe is df and column to replace is 'protein_symbol'
df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].apply(replace_with_base_name)


df_melted_protein = df_melted_protein[df_melted_protein['protein_symbol'].isin(montagud_nodes)]
df_melted_protein['protein_symbol'] = df_melted_protein['protein_symbol'].str.replace('_', '', regex=False)
df_melted_protein = df_melted_protein[df_melted_protein['rsem_tpm'].notna()]
df_melted_protein.to_csv('data/TCGA_data/prostate/filtered_data/proteins_samples_table.csv')


In [9]:
# Pre-processing CNV data (only id in the patients id list and proteins in the montagud list)

cnv_data_col = list(cnv_data.columns)
common_col = list(set(cnv_data_col) & set(patients_id))
col_keep = ['Gene Symbol'] + common_col
cnv_data_filtered = cnv_data[col_keep]

df_melted_cnv = cnv_data_filtered.melt(
    id_vars=["Gene Symbol"],       # columns to keep fixed
    var_name="samples_id",         # name for the variable column (sample IDs)
    value_name="expression_value"  # name for the values
)


df_melted_cnv['Gene Symbol'] = df_melted_cnv['Gene Symbol'].str.split('|').str[0] 


df_melted_cnv = df_melted_cnv.rename(
    columns={
        "samples_id": "model_id",
        "Gene Symbol": "gene_symbol",
        "expression_value": "rsem_tpm",
    }
)

group_loss = [-1, -2]
group_normal = [0]
group_gain = [1, 2]

conditions = [
    df_melted_cnv["rsem_tpm"].isin(group_loss),
    df_melted_cnv["rsem_tpm"].isin(group_normal),
    df_melted_cnv["rsem_tpm"].isin(group_gain),
]
choices = ["Loss", "Normal", "Gain"]
df_melted_cnv.loc[:, "effect"] = np.select(conditions, choices, default="")

df_melted_cnv = df_melted_cnv[df_melted_cnv['gene_symbol'].isin(montagud_nodes)]
df_melted_cnv.to_csv('data/TCGA_data/prostate/filtered_data/cnv_samples_table.csv')

In [10]:
# Read the two processed data
df_melted_cnv= pd.read_csv('data/TCGA_data/prostate/filtered_data/cnv_samples_table.csv')
def_melted_proteins = pd.read_csv('data/TCGA_data/prostate/filtered_data/proteins_samples_table.csv')
def_melted_proteins = def_melted_proteins[['protein_symbol', 'model_id', 'rsem_tpm']]

In [11]:
 # Create generic models 

folder_generic_models_cfg = f'{folder_generic_models}/Montagud2022_Prostate_Cancer.cfg'
folder_generic_models_bnd = f'{folder_generic_models}/Montagud2022_Prostate_Cancer.bnd'

create_generic_patients_cfg_bnd_validation(folder_generic_models_cfg, folder_generic_models_bnd, folder_pers_models, patients_id, tissue)


All .cfg and .bnd files created for the validation.


In [12]:
# update phenotypes in generic models 

# original_data_dir = "validation/prostate/personalized_models/proteins_models"
# results_dir = "validation/prostate/personalized_models/proteins_models"


# generic_models_update_phenotypes(phenotype_interest, original_data_dir, results_dir)
generic_models_update_phenotypes(phenotype_interest, folder_pers_models, folder_pers_models)


Updated BIRC5.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated GLUT1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated CASPASE8.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated TSC1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated GSH.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated ROS.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated APAF1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated WNT.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated PDK1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated PTCH1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated RAF.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated NCOR1.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated MEK2.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated P38.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated VHL.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated EZH2.is_internal=1 in TCGA-ZG-A9L5-01_Prostate.cfg
Updated Invasion.is_internal=0 in TCGA-ZG-A9L5-01_Pro

In [13]:
# personalize the boolean networks with genes 
table_proteins_patients = create_table_proteins_patients(def_melted_proteins)

/Users/romane/repos/icr/Attractor-Resistance/pre_process_data/pre_process_genes.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proteins_data.rename(columns={"protein_symbol": "protein_name"}, inplace=True)


In [14]:
personalized_patients_proteins_cfgs(df_melted_protein,montagud_nodes,folder_pers_models,folder_pers_models,patients_id,table_proteins_patients,tissue)

Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-ZG-A9L5-01_Prostate.cfg
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-EJ-A7NN-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-G9-6343-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-CH-5743-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-YL-A9WI-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-YL-A8SF-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-Y6-A8TL-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-V1-A8MF-01_Prostate.bnd
Modified and saved: validation/prostate/proteins_models/personalized_models/TCGA-YL-A8S8-01_Prostate.cfg
Modified and saved: validation/prostate/proteins_models

In [15]:
# personalize with CNV
tailor_bnd_cnv_validation(df_melted_cnv, folder_pers_models, tissue)

🔍 Processing patient TCGA-HC-7736-01, gene: ZBTB17
ZBTB17 node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: JUN
JUN node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: PDK1
PDK1 node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: CFLAR
CFLAR node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: IDH1
IDH1 node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: VHL
VHL node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: ATR
ATR node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: MAP3K13
No MAP3K13 node found in file for patient TCGA-HC-7736-01
🔍 Processing patient TCGA-HC-7736-01, gene: FGFR3
FGFR3 node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: EGF
EGF node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: TERT
TERT node found. Replacing...
🔍 Processing patient TCGA-HC-7736-01, gene: DAXX
DAXX node found. Replacing...
🔍 Processing patient 

Let's try to add more info to reflect metastasis change, add mutations data

In [16]:

phenotypes_interest = [
    "Proliferation",
    "Invasion",
    "DNA_Repair",
    "Migration",
    "Apoptosis",
]

inputs_list = [
    "EGF",
    "FGF",
    "TGFB",
    "Androgen",
    "Hypoxia",
    "Nutrients",
    "Carcinogen",
    "Acidosis",
    "TNF",
    "fused_event",
    "SPOP",
]
for patient in patients_id:
    compute_phenotype_table(folder_save_results,folder_pers_models,patient,inputs_list,phenotypes_interest,tissue="Prostate")


In [17]:
low_group_ids= list(phenotype_data_filtered[phenotype_data_filtered['Gleason_group'] == 'low_aggressive']['sampleID'])
medium_group_ids= list(phenotype_data_filtered[phenotype_data_filtered['Gleason_group'] == 'middle_aggressive']['sampleID'])
high_group_ids= list(phenotype_data_filtered[phenotype_data_filtered['Gleason_group'] == 'high_aggressive']['sampleID'])

In [18]:
#move each files to directory corresponding

# Map group names to sample ID lists
group_mapping = {
    "low_group": low_group_ids,
    "medium_group": medium_group_ids,
    "high_group": high_group_ids,
}

# Loop over all files in the source directory
for filename in os.listdir(folder_save_results):
    if not filename.startswith("_TCGA"):
        continue
    sample_id = filename.replace("_", "").replace(".csv", "")
    # Determine the group of this sample
    group_found = False
    for group_name, id_list in group_mapping.items():
        if sample_id in id_list:
            group_folder = os.path.join(dest_base_dir, group_name)
            os.makedirs(group_folder, exist_ok=True)

            src_path = os.path.join(folder_save_results, filename)
            dst_path = os.path.join(group_folder, filename)
            shutil.move(src_path, dst_path)

            group_found = True
            break

    if not group_found:
        print(f" Sample ID {sample_id} not found in any group list.")


In [19]:
# combine all the values
groups = ["low_group", "medium_group", "high_group"]

mean_df =compute_phenotype_mean_group_validation(groups, dest_base_dir)

              Proliferation  Invasion  DNA_Repair  Migration  Apoptosis
Acidosis           0.290579  0.050721    0.232938   0.009585   0.084828
Androgen           0.293615  0.068617    0.233018   0.034497   0.081915
Carcinogen         0.325627  0.086237    0.438683   0.009557   0.187523
EGF                0.309252  0.077208    0.234981   0.024735   0.108935
FGF                0.320254  0.067901    0.233618   0.009629   0.144708
Hypoxia            0.247509  0.046289    0.234267   0.015461   0.072850
Nutrients          0.340933  0.051119    0.232867   0.012736   0.060943
SPOP               0.324251  0.103437    0.234647   0.025701   0.051520
TGFB               0.250197  0.278431    0.233018   0.023529   0.235610
TNF                0.258199  0.305083    0.232839   0.062228   0.067149
fused_event        0.286144  0.051121    0.232977   0.012672   0.071255
Overall_Mean       0.295142  0.107833    0.252168   0.021848   0.106112
              Proliferation  Invasion  DNA_Repair  Migration  Ap

In [20]:
# combine values of a directory together
def collect_group_data(group_folder_path):
    combined_data = defaultdict(lambda: defaultdict(list))

    for file in os.listdir(group_folder_path):
        if file.startswith("_TCGA") and file.endswith(".csv"):
            file_path = os.path.join(group_folder_path, file)
            df = pd.read_csv(file_path, index_col=0)

            for input_name in df.index:
                for phenotype in df.columns:
                    value = df.at[input_name, phenotype]
                    combined_data[input_name][phenotype].append(float(value))

    result_df = pd.DataFrame.from_dict(combined_data, orient='index')
    result_df.to_csv(os.path.join(group_folder_path, "combined_results.csv"))

    return result_df


group_names = ["low_group", "medium_group", "high_group"]
group_dataframes = {}

for group in group_names:
    folder_path = os.path.join(dest_base_dir, group)
    group_df = collect_group_data(folder_path)
    group_dataframes[group] = group_df
group_dataframes

{'low_group':                                                  Proliferation  \
 EGF          [0.016, 0.988, 0.026595, 0.271145, 0.03, 0.913...   
 FGF          [0.022, 0.970758, 0.025557, 0.312513, 0.021623...   
 TGFB         [0.0, 0.965461, 0.038, 0.0547869999999999, 0.0...   
 Androgen     [0.022, 0.970758, 0.025557, 0.212161, 0.021623...   
 Hypoxia      [0.012045, 0.98, 0.014, 0.044189, 0.0272309999...   
 Nutrients    [0.017208, 0.994, 0.0199999999999999, 0.413065...   
 Carcinogen   [0.03, 0.979728, 0.026, 0.346861, 0.026, 0.930...   
 Acidosis     [0.006, 0.970758, 0.028, 0.1714709999999999, 0...   
 TNF          [0.019054, 0.969113, 0.023714, 0.086129, 0.019...   
 fused_event  [0.031921, 0.982, 0.042561, 0.155647, 0.034455...   
 SPOP         [0.012877, 0.994508, 0.02, 0.34937, 0.01, 0.88...   
 
                                                       Invasion  \
 EGF          [0.006, 0.008593, 0.016, 0.2258129999999999, 0...   
 FGF          [0.024, 0.016, 0.026, 0.157595, 0

In [21]:
# Stats test- Kruskal test

# Paths to your combined data CSVs
group_files = {
    "low": os.path.join(dest_base_dir, "low_group", "combined_results.csv"),
    "medium": os.path.join(dest_base_dir, "medium_group", "combined_results.csv"),
    "high": os.path.join(dest_base_dir, "high_group", "combined_results.csv"),
}

# Load all groups into dict of DataFrames
group_dfs = {}
for group, path in group_files.items():
    # Because each cell is a list saved as a string, parse it back to list
    df = pd.read_csv(path, index_col=0)
    # Convert strings like '[1.2, 3.4]' back to Python lists using ast.literal_eval
    df = df.applymap(ast.literal_eval)
    group_dfs[group] = df

# Get all inputs and phenotypes from one dataframe (assuming all share the same shape)
inputs = group_dfs["low"].index
phenotypes = group_dfs["low"].columns

# Prepare result storage
kruskal_results = pd.DataFrame(index=inputs, columns=phenotypes)

# Run Kruskal-Wallis test for each (input, phenotype)
for input_name in inputs:
    for phenotype in phenotypes:
        data_low = group_dfs["low"].at[input_name, phenotype]
        data_medium = group_dfs["medium"].at[input_name, phenotype]
        data_high = group_dfs["high"].at[input_name, phenotype]

        # Run the Kruskal-Wallis test only if all groups have data
        if data_low and data_medium and data_high:
            stat, pvalue = kruskal(data_low, data_medium, data_high)
            kruskal_results.at[input_name, phenotype] = pvalue
        else:
            kruskal_results.at[input_name, phenotype] = None

# Optionally, save the p-values table to CSV
kruskal_results.to_csv(os.path.join(dest_base_dir, "kruskal_pvalues.csv"))


/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_27579/654129326.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(ast.literal_eval)
/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_27579/654129326.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(ast.literal_eval)
/var/folders/83/pqwsm5cn7vscbq8l3fnf30qr0000gn/T/ipykernel_27579/654129326.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(ast.literal_eval)


In [22]:

# Flatten p-values to a 1D array, ignoring None or NaNs
pvals = kruskal_results.values.flatten()
pvals = [p for p in pvals if p is not None]

# Adjust using BH method
_, pvals_adj, _, _ = multipletests(pvals, alpha=0.05, method='fdr_bh')

# Now, you need to put adjusted p-values back into the DataFrame shape
# Create a copy to fill
adjusted_df = kruskal_results.copy()

# Fill adjusted p-values sequentially where there was a non-None p-value
idx = 0
for i in adjusted_df.index:
    for j in adjusted_df.columns:
        if adjusted_df.at[i, j] is not None:
            adjusted_df.at[i, j] = pvals_adj[idx]
            idx += 1

In [23]:
# keep only the significant results
significant_df = adjusted_df.copy()
significant_df[significant_df >= 0.05] = np.nan
significant_df

,Proliferation,Invasion,DNA_Repair,Migration,Apoptosis
EGF,0.011421,NaN,0.010287,NaN,NaN
FGF,0.004603,NaN,0.003973,NaN,NaN
TGFB,0.002707,NaN,0.001306,0.00672,NaN
Androgen,0.003973,NaN,0.006997,NaN,NaN
Hypoxia,0.002707,NaN,0.00525,NaN,NaN
Nutrients,0.016258,NaN,0.002707,NaN,NaN
Carcinogen,0.009707,NaN,0.013145,NaN,NaN
Acidosis,0.004603,NaN,0.003704,NaN,NaN
TNF,0.009707,NaN,0.001306,NaN,NaN
fused_event,0.011421,NaN,0.003704,NaN,NaN
